# HW Group A 13

## 2. Mean-Variance Optimization

In [36]:
import pandas as pd
import numpy as np
import csv
import math
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [5]:
price = pd.read_excel('multi_asset_etf_data.xlsx',sheet_name='prices', index_col=0)
TR = pd.read_excel('multi_asset_etf_data.xlsx',sheet_name='total returns', index_col=0)
ER = pd.read_excel('multi_asset_etf_data.xlsx',sheet_name='excess returns', index_col=0)

In [6]:
ER.head()

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2009-04-30,0.008441,-0.001553,0.155030,0.114637,0.137907,-0.028004,0.295598,0.229650,0.022329,0.098793,-0.018505
2009-05-31,0.054143,0.163134,0.159871,0.132389,0.029026,-0.020303,0.023198,0.054363,0.028336,0.058925,0.020437
2009-06-30,0.004550,-0.026858,-0.023094,-0.014648,0.032919,-0.006170,-0.025462,0.041443,-0.004035,-0.001254,0.001383
2009-07-31,0.031312,0.018595,0.110173,0.100442,0.069217,0.008344,0.105826,0.143274,0.015353,0.074633,0.000906
2009-08-31,0.007193,-0.040800,-0.013571,0.044595,-0.017404,0.007199,0.131504,0.032977,-0.004586,0.036504,0.007979


## (1) Summary Statistics

### (a) Calculate and display the mean and volatility of each asset’s excess return. (Recall we use volatility to refer to standard deviation.)

$\bullet$ Assumptions regarding annualized returns and volatility, drawn from historical data for each asset, form the basis of the analysis.

In [28]:
def summary_stats(df, annualization_factor):
    mean = df.mean() * annualization_factor
    volatility = df.std() * np.sqrt(annualization_factor)
    sharpe = mean / volatility
    return pd.DataFrame({'Mean': round(mean, 4), 'Volatility': round(volatility, 4), 'Sharpe Ratio': round(sharpe, 4)})

In [29]:
summary_statistics = summary_stats(ER, 12)
summary_statistics.transpose()

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Mean,-0.0018,0.0254,0.0649,0.0816,0.0642,0.0143,0.1295,0.0799,0.0190,0.1437,0.0223
Volatility,0.0834,0.1790,0.1965,0.1660,0.0892,0.0624,0.1871,0.2274,0.0508,0.1477,0.0515
Sharpe Ratio,-0.0221,0.1422,0.3302,0.4916,0.7197,0.2287,0.6920,0.3516,0.3734,0.9732,0.4332


### (b) Which assets have the best and worst Sharpe ratios? Recall that the Sharpe Ratio is simply the ratio of the mean-to-volatility of excess returns:

$\bullet$ The asset with the best Sharpe Ratio is SPY with a Sharpe ratio of 0.9732, and the asset with the worst Sharpe Ratio is BWX with a Sharpe ratio of -0.0221.

$\bullet$ SPY's broad diversification, mirroring a diversified equity index, contributes to a balanced risk-return profile. Conversely, BWX’s specialized focus on foreign sovereign debt, constrained by index tracking, exposes it to specific risks such as currency and country-specific risks, affecting its Sharpe ratio negatively.

$\bullet$ The differing asset classes—equities for SPY and sovereign debt for BWX—result in distinct risk-return profiles. Historically, equities offer higher return potential though with increased volatility, while sovereign bonds are perceived as lower-risk with lesser return prospects.

In [33]:
max_SR = summary_statistics.nlargest(1, 'Sharpe Ratio')
min_SR = summary_statistics.nsmallest(1, 'Sharpe Ratio')
pd.concat([max_SR,min_SR]).loc[:,['Mean','Volatility','Sharpe Ratio']].transpose()

,SPY,BWX
Mean,0.1437,-0.0018
Volatility,0.1477,0.0834
Sharpe Ratio,0.9732,-0.0221


## (2) Descriptive Analysis

### (a) Calculate the correlation matrix of the returns. Which pair has the highest correlation? And the lowest?

In [40]:
corr_mat = round(ER.corr(), 4)
corr_mat

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
BWX,1.0000,0.3498,0.6476,0.6217,0.5577,0.4345,0.4535,0.5249,0.6680,0.4657,0.6171
DBC,0.3498,1.0000,0.5657,0.5819,0.4732,-0.3217,0.3183,0.4961,0.5479,0.5099,0.1367
EEM,0.6476,0.5657,1.0000,0.8516,0.7260,-0.1023,0.6218,0.7717,0.8072,0.7346,0.3027
EFA,0.6217,0.5819,0.8516,1.0000,0.7715,-0.1323,0.6979,0.8919,0.8537,0.8716,0.2875
HYG,0.5577,0.4732,0.7260,0.7715,1.0000,-0.0086,0.7576,0.8238,0.7688,0.7704,0.3659
IEF,0.4345,-0.3217,-0.1023,-0.1323,-0.0086,1.0000,0.0736,-0.1187,0.0557,-0.1557,0.7061
IYR,0.4535,0.3183,0.6218,0.6979,0.7576,0.0736,1.0000,0.7602,0.6560,0.7536,0.3972
PSP,0.5249,0.4961,0.7717,0.8919,0.8238,-0.1187,0.7602,1.0000,0.8383,0.8957,0.3209
QAI,0.6680,0.5479,0.8072,0.8537,0.7688,0.0557,0.6560,0.8383,1.0000,0.8410,0.4597
SPY,0.4657,0.5099,0.7346,0.8716,0.7704,-0.1557,0.7536,0.8957,0.8410,1.0000,0.2946


$\bullet$ The pair $\textbf {SPY}$ and $\textbf {PSP}$ has the $\textbf {highest}$ correlation of $\textbf {0.8957}$.

$\bullet$ The high correlation between SPY and PSP likely stems from their shared sensitivity to market movements, economic conditions, and investor sentiment. Both ETFs, although representing different market segments, are traded on recognized exchanges and can react similarly to overall market trends and liquidity conditions, causing their performances to mirror each other.

$\bullet$ The pair $\textbf {DBC}$ and $\textbf {IEF}$ has the $\textbf {lowest}$ correlation of $\textbf {-0.3217}$.

$\bullet$ The low correlation between DBC and IEF arises from their distinct asset classes and market influences. DBC, a commodity-focused fund, is affected by supply-demand dynamics and global economic conditions, while IEF, investing in U.S. Treasury securities, is influenced by U.S. monetary policy and interest rates. These differing market exposures lead to a divergence in their performance trends, resulting in a low correlation.

In [184]:
corr = ER.corr()
cov=ER.cov()
stacked_corr = corr.stack()
highest_corr_pairs = stacked_corr.nlargest(12)
lowest_corr_pairs = stacked_corr.nsmallest(5)
print('Highest correlation pair is ',highest_corr_pairs.index[11])
print('Lowest correlation pair is ',lowest_corr_pairs.index[0])

Highest correlation pair is  ('PSP', 'SPY')
Lowest correlation pair is  ('DBC', 'IEF')


### (b) How well have TIPS done in our sample? Have they outperformed domestic bonds? Foreign bonds?

$\bullet$ $\textbf {Yes}$, The data provided indicates that Treasury Inflation-Protected Securities (TIPS) have had a better performance with a mean return of 0.0223 compared to domestic bonds (IEF) with a 0.0143 mean return, and foreign bonds (BWX) which had a negative mean return of -0.0018. This suggests that TIPS were the more profitable investment over the analyzed period.

$\bullet$ Moreover, TIPS exhibited lower volatility (0.0515) compared to IEF (0.0624) and BWX (0.0834), indicating lower risk. The Sharpe Ratio further underscores this by showing TIPS had a better risk-adjusted performance of 0.4332, compared to 0.2287 for domestic bonds and -0.0221 for foreign bonds.

In [46]:
summary_statistics.loc[['TIP', 'IEF', 'BWX']].transpose()

,TIP,IEF,BWX
Mean,0.0223,0.0143,-0.0018
Volatility,0.0515,0.0624,0.0834
Sharpe Ratio,0.4332,0.2287,-0.0221


In [186]:
#calculate W_Tan based on fomulas
W_tan=(1/(np.transpose(np.ones(11))@np.linalg.inv(cov)@mean))*np.linalg.inv(cov)@mean
print('W_tan=',W_tan)

W_tan= [-1.46497414  0.02843646  0.26102769  0.45291399  1.52894242  1.89399165
 -0.24277199 -1.2710546  -3.13344544  2.58999871  0.35693524]


In [187]:
print('The W_tan aligns perfectly with Sharpe ratio of each asset')

The W_tan aligns perfectly with Sharpe ratio of each asset


In [194]:
mean_W_tan=np.transpose(W_tan)@mean
vol_W_tan=math.sqrt(np.transpose(W_tan)@cov@W_tan)
SP_W_tan=mean_W_tan/vol_W_tan
print('Mean of W_tan=',mean_W_tan)
print('Volatility of W_tan=',vol_W_tan)
print('Sharpe Ratio of W_tan=',SP_W_tan)

Mean of W_tan= 0.37531992061454567
Volatility of W_tan= 0.05528789276868884
Sharpe Ratio of W_tan= 6.788464920969106


In [195]:
#We drop Tip in ER
ER1=ER.drop('TIP', axis=1)
W_tan1=(1/(np.transpose(np.ones(10))@np.linalg.inv(ER1.cov()*math.sqrt(12))@ER1.mean()*12))*np.linalg.inv(ER1.cov()*math.sqrt(12))@ER1.mean()*12
print(W_tan1)
mean_W_tan1=np.transpose(W_tan1)@ER1.mean()*12
vol_W_tan1=math.sqrt(np.transpose(W_tan1)@ER1.cov()*math.sqrt(12)@W_tan1)
SP_W_tan1=mean_W_tan1/vol_W_tan1
print('Mean of W_tan without TIP=',mean_W_tan1)
print('Volatility of W_tan without TIP=',vol_W_tan1)
print('Sharpe Ratio of W_tan without TIP=',SP_W_tan1)

[-1.51274965  0.05515758  0.27808582  0.44149597  1.5931398   2.21245096
 -0.24589531 -1.31409179 -3.23895589  2.73136252]
Mean of W_tan without TIP= 0.3862908162901122
Volatility of W_tan without TIP= 0.10751662610569362
Sharpe Ratio of W_tan without TIP= 3.592847267271679


In [196]:
ER['TIP']+=0.0012
W_tan2=(1/(np.transpose(np.ones(11))@np.linalg.inv(ER.cov()*math.sqrt(12))@ER.mean()*12))*np.linalg.inv(ER.cov()*math.sqrt(12))@ER.mean()*12
print(W_tan2)
mean_W_tan2=np.transpose(W_tan2)@ER.mean()*12
vol_W_tan2=math.sqrt(np.transpose(W_tan2)@ER.cov()*math.sqrt(12)@W_tan2)
SP_W_tan2=mean_W_tan2/vol_W_tan2
print('Mean of W_tan adjusting TIP=',mean_W_tan2)
print('Volatility of W_tan adjusting TIP=',vol_W_tan2)
print('Sharpe Ratio of W_tan adjusting TIP=',SP_W_tan2)

[-1.12679503 -0.16070899  0.14028166  0.53373651  1.07452112 -0.36022334
 -0.22066354 -0.96641575 -2.38658944  1.5893546   2.8835022 ]
Mean of W_tan adjusting TIP= 0.3391848411984788
Volatility of W_tan adjusting TIP= 0.08082046831461605
Sharpe Ratio of W_tan adjusting TIP= 4.196769064466539
